# Comma. ai  <br>


## JOH EUN KANG (Summer Internship : ML software engineer)
##  Code & Comments


# Directory

In [582]:
%cd /Users/joheunkang/Desktop/comma/speed_challenge_2017/data

/Users/joheunkang/Desktop/comma/speed_challenge_2017/data


# Libraries

In [739]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os, time, sys, shutil
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers import Flatten, Dense, Lambda, Convolution2D, Cropping2D, Dropout, Reshape, BatchNormalization, ELU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image


# Defining Paths

In [777]:
# Your datat location
PATH_DATA_FOLDER = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data'
PATH_TRAIN_LABEL = os.path.join(data_path, 'train.txt')
PATH_TRAIN_VIDEO = os.path.join(data_path, 'train.mp4')

PATH_TEST_VIDEO = os.path.join(data_path, 'test.mp4')
PATH_TEST_VIDEO_OUTPUT = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/test_video_output'
PATH_COMBINED_TEST_VIDEO_OUTPUT = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/comb_test_video_output'

OUTPUT_TRAIN_PATH = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_train_img'
OUTPUT_TEST_PATH = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_test_img'

processed_train_imgs = os.path.join(processed_data_path, 'train_imgs')
processed_test_imgs = os.path.join(processed_data_path, 'test_imgs')
DOWNLOAD_PATH = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/final_result'

# your final predicted speed result 
test_labels_result = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/results/test_lables.txt'

# Converting frames to Optical Flow

After converting your video to frames, we are again converting those original images to optical flow images.
Since opencv usese BGR as difault, we change BGR to gray.<br>
Then, using the __calcOpticalFlowFarneback__, we get the image of optical flow. 

In [680]:
def convertToOptical(prev_image, curr_image):
    prev_image_gray = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)
    curr_image_gray = cv2.cvtColor(curr_image, cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prev_image_gray, curr_image_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    hsv = np.zeros_like(prev_image)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    
    # first, second, and third chnnels
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,1] = 255
    hsv[...,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    flow_image_bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return flow_image_bgr

# Get Optical flow from the Video

Here, we change those images to optical flow using the opticalflow definition,

In [681]:
OUTPUT_FLOW_TRAIN_PATH = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img'
OUTPUT_FLOW_TEST_PATH = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_test_img'
OUTPUT_FLOW_VIDEO_TRAIN_PATH = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_vid'
OUTPUT_FLOW_VIDEO_TEST_PATH= '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_test_vid'
DOWNLOAD_PATH = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/final_result'


def preprocess_data(video_input_path, flow_video_output_path, image_folder_path, flow_image_folder_path, type):
    
    #make frame img
    if os.path.exists(image_folder_path):
        shutil.rmtree(image_folder_path)
    os.makedirs(image_folder_path)
    
    # make optical flow img
    if os.path.exists(flow_image_folder_path):
        shutil.rmtree(flow_image_folder_path)
    os.makedirs(flow_image_folder_path)

    print("Converting video to optical flow for: ", video_input_path)

    video_reader = cv2.VideoCapture(video_input_path)

    num_frames = video_reader.get(cv2.CAP_PROP_FRAME_COUNT)
    frame_size = (int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fps = int(video_reader.get(cv2.CAP_PROP_FPS))

    video_writer = cv2.VideoWriter(flow_video_output_path, fourcc, fps, frame_size)

    t1 = time.time()
    ret, prev_frame = video_reader.read()
    hsv = np.zeros_like(prev_frame)

    image_path_out = os.path.join(image_folder_path, str(0) + '.jpg')
    cv2.imwrite(image_path_out, prev_frame)

    count = 1
    while True:
        ret, next_frame = video_reader.read()
        if next_frame is None:
            break

        bgr_flow = convertToOptical(prev_frame, next_frame)

        image_path_out = os.path.join(image_folder_path, str(count) + '.jpg')
        flow_image_path_out = os.path.join(flow_image_folder_path, str(count)+'flow' + '.jpg')

        cv2.imwrite(image_path_out, next_frame)
        cv2.imwrite(flow_image_path_out, bgr_flow)

        video_writer.write(bgr_flow)

        prev_frame = next_frame
        count += 1

        
    t2 = time.time()
    video_reader.release()
    video_writer.release()
    print(' Conversion completed !')
    print(' Time Taken:', (t2 - t1), 'seconds')

    
    return

In [67]:
preprocess_data(PATH_TRAIN_VIDEO, OUTPUT_FLOW_VIDEO_TRAIN_PATH, OUTPUT_TRAIN_PATH, OUTPUT_FLOW_TRAIN_PATH, type='train')


Converting video to optical flow for:  /Users/joheunkang/Desktop/comma/speed_challenge_2017/data/train.mp4
 Conversion completed !
 Time Taken: 1978.3607580661774 seconds


In [68]:
preprocess_data(PATH_TEST_VIDEO, OUTPUT_FLOW_VIDEO_TEST_PATH, OUTPUT_TEST_PATH, OUTPUT_FLOW_TEST_PATH, type='test')


Converting video to optical flow for:  /Users/joheunkang/Desktop/comma/speed_challenge_2017/data/test.mp4
 Conversion completed !
 Time Taken: 938.9039421081543 seconds


__DONE WITH THE DATA PRE-PROCESSING__

In [736]:
TYPE_FLOW_PRECOMPUTED = 0
TYPE_ORIGINAL = 1

# Model

we use CNN Model with ELU, with 5 convolutional layers.<br>

The CNN Model looks simple but it is one of the most popular model in the Computer Vision! 



In [742]:
def CNNModel():
    model = Sequential()
    model.add(Convolution2D(24, (5, 5),
                            input_shape = (240, 320, 3),
                            strides=(2,2),
                            padding = 'valid',
                            kernel_initializer = 'he_normal'))
    model.add(ELU())
    model.add(Convolution2D(36, (5, 5),
                            strides=(2,2),
                            padding = 'valid',
                            kernel_initializer = 'he_normal'))
    model.add(ELU())
    model.add(Convolution2D(48, (5, 5),
                            strides=(2,2),
                            padding = 'valid',
                            kernel_initializer = 'he_normal'))
    model.add(ELU())
    model.add(Dropout(0.5))
    model.add(Convolution2D(64, (3, 3),
                            strides = (1,1),
                            padding = 'valid',
                            kernel_initializer = 'he_normal'))
    model.add(ELU())
    model.add(Convolution2D(64, (3, 3),
                            strides= (1,1),
                            padding = 'valid',
                            kernel_initializer = 'he_normal'))
    model.add(Flatten())
    model.add(ELU())
    
    model.add(Dense(100, kernel_initializer = 'he_normal'))
    model.add(ELU())
    model.add(Dense(50, kernel_initializer = 'he_normal'))
    model.add(ELU())
    model.add(Dense(10, kernel_initializer = 'he_normal'))
    model.add(ELU())
    model.add(Dense(1, kernel_initializer = 'he_normal'))

    adam = Adam(lr=1e-4)
    model.compile(optimizer=adam, loss="mse")

    return model

# Define 

In [743]:
type_ = TYPE_FLOW_PRECOMPUTED
TYPE_FLOW_PRECOMPUTED = 0

# Prepare Data (modified_version)

So far, we have images from original training video, and the optical flow images.
In this part, we are going to have the training images pairs by using one original image, and the __four consecutive optical flow images__.  <br>

Each training image pair will have 5 images and the a single speed that is used in the original image<br>

```python

'/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_train_img/6.jpg'
'/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/3flow.jpg'
'/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/4flow.jpg'
'/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/5flow.jpg'
'/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/6flow.jpg'
```

so, the speed for this pair will be the speed of image 6.jpg <br>



In [ ]:
# Prepare data example

def prepareData(labels_path, images_path, flow_images_path, type=TYPE_FLOW_PRECOMPUTED):
    num_train_labels = 0
    train_labels = []
    train_images_pair_paths = []

    with open(labels_path) as txt_file:
        labels_string = txt_file.read().split()

        for i in range(4, len(labels_string)):
            speed = float(labels_string[i])
            train_labels.append(speed)

            if type == TYPE_FLOW_PRECOMPUTED:
                train_images_pair_paths.append((os.path.join(images_path,np.str(i)+'.jpg'),
                                                             os.path.join(flow_images_path,np.str(i-3)+'flow.jpg')
                                                                ,os.path.join(flow_images_path,np.str(i-2)+'flow.jpg')
                                                                    ,os.path.join(flow_images_path,np.str(i-1)+'flow.jpg')
                                                                        ,os.path.join(flow_images_path,np.str(i)+'flow.jpg')))
           
    return train_images_pair_paths, train_labels

          
          
          

__From this function, we get one original train image , 4 consecutive flow image with the speed of original train image.__ <br>



In [745]:
train_images_pair_path, train_labels = prepareData(PATH_TRAIN_LABEL, OUTPUT_TRAIN_PATH ,OUTPUT_FLOW_TRAIN_PATH, type = type_)


In [746]:
train_images_pair_path[2]

('/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_train_img/6.jpg',
 '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/3flow.jpg',
 '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/4flow.jpg',
 '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/5flow.jpg',
 '/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/6flow.jpg')

In [747]:
train_labels[2]

28.034211

#  Training Samples and Testing Samples

After we have the whole traning pairs, we are going to split the training paris to make new traning set and validation set. <br>
Since we don't have the labels( "speed") for our testing video ("test set"), __we need to validate our model using validation data.__<br>


In [748]:
samples = list(zip(train_images_pair_path,train_labels))
train_samples, validation_samples = train_test_split(samples,test_size = 0.2)

In [749]:
len(train_samples)

16316

In [750]:
len(validation_samples)

4080

In [751]:
print('Total Images: {}'.format( len(train_images_pair_paths)))

Total Images: 20396


In [752]:
print('Train samples: {}'.format(len(train_samples)))
print('Validation samples: {}'.format(len(validation_samples)))


Train samples: 16316
Validation samples: 4080


# Generate Data

This is the __key part__ of our challenge.<br>

from the training and validation set, we are going to have the combined image of the optical flow images. 
In the previous part, we have made the trainig and validation set with a single original image and four corresponing optical flow images.<br>

Here, I decided to combine those four optical flow images to make a new single image. We may use 3 or 5 or more images. However, if you combine too many images for making a new single image, you might have some troubles.<br>

Assume you have used two images. This means the model will have the optical flow with only two frames.<br>
If the car moves very fast, then this could be better choice than 4 images. However, as you can see, the car in the video is not moving very fast. So if you take 2 images, the data may not tell enough information of the current speed.

Also, we are augmenting some more images that are flipped version of the optical flow images.<br> In order to have more tranining data, we may add some __light changed images__ or __rotated images.__ I have trained with the light changed images, but the result was not good. Also, if we want to augment rotated images of optical flow, we need to rotate the whole images with several angles ( lot of work!) (like MNIST rotated data set). 


##### batch sample visualization
batch_samples (('/Users/joheunkang/Desktop/comma/speed_challenge_2017/dataUsers/joheunkang/Desktop/comma/speed_challenge_2017/data/new_train_img/3047.jpg', '/Users/joheunkang/Desktop/comma/speed_challenge_2017/dataUsers/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/3044flow.jpg', '/Users/joheunkang/Desktop/comma/speed_challenge_2017/dataUsers/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/3045flow.jpg', '/Users/joheunkang/Desktop/comma/speed_challenge_2017/dataUsers/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/3046flow.jpg', '/Users/joheunkang/Desktop/comma/speed_challenge_2017/dataUsers/joheunkang/Desktop/comma/speed_challenge_2017/data/new_flow_train_img/3047flow.jpg'), 21.939557)

In [754]:
def generatorData_non(samples, batch_size=32, type=TYPE_FLOW_PRECOMPUTED):
    num_samples = len(samples)
    while True:
        samples = sklearn.utils.shuffle(samples)
        #print('define batch_size:',batch_size)
        for offset in range(0, num_samples, batch_size):
            #print('offset',offset)
            batch_samples = samples[offset:offset+batch_size]
            #print('len_batch_size',len(batch_samples))
            #print(batch_samples[0])
            images = []
            angles = []
            for imagePath, measurement in batch_samples:
                combined_image = None
                flow_image_bgr = None
            
                if type ==TYPE_FLOW_PRECOMPUTED:
                    curr_image_path, flow_image_path1, flow_image_path2,flow_image_path3, flow_image_path4 = imagePath
                    flow_image_bgr = (cv2.imread(flow_image_path1) +cv2.imread(flow_image_path2) +cv2.imread(flow_image_path3) +cv2.imread(flow_image_path4) )/4
                
                    curr_image = cv2.imread(curr_image_path)
                    curr_image = cv2.cvtColor(curr_image, cv2.COLOR_BGR2RGB)
                
                    # combined_image = 0.1*curr_image + flow_image_bgr
                    #CHOOSE IF WE WANT TO TEST WITH ONLY OPTICAL FLOW OR A COMBINATION OF VIDEO AND OPTICAL FLOW
                    combined_image = flow_image_bgr
                
                    combined_image = cv2.normalize(combined_image, None, alpha=-1, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                    combined_image = cv2.resize(combined_image, (0,0), fx=0.5, fy=0.5)
                
                    images.append(combined_image)
                    angles.append(measurement)
                
                    # Adding more images!
                    # after we combined the image, we are going to add the flipped images 
                    images.append(cv2.flip(combined_image,1))
                    angles.append(measurement)
                
                inputs = np.array(images)
                outputs = np.array(angles)
                yield sklearn.utils.shuffle(inputs, outputs)
        

# Constructing the generator

with the generator above, we generate traning and valid data set with batch size we assinged.

In [755]:
training_generator = generatorData_non(train_samples, batch_size=BATCH_SIZE, type=type_)
validation_generator = generatorData_non(validation_samples, batch_size=BATCH_SIZE, type=type_)


In [756]:
print('/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_train_img/4647.jpg')

/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_train_img/4647.jpg


In [757]:
a = cv2.imread('/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_train_img/4647.jpg')

In [758]:
a

array([[[ 8,  3,  2],
        [ 8,  3,  2],
        [ 8,  3,  2],
        ...,
        [ 1,  1,  1],
        [ 1,  1,  1],
        [ 1,  1,  1]],

       [[10,  5,  4],
        [10,  5,  4],
        [10,  5,  4],
        ...,
        [ 1,  1,  1],
        [ 1,  1,  1],
        [ 1,  1,  1]],

       [[13,  8,  7],
        [13,  8,  7],
        [13,  8,  7],
        ...,
        [ 1,  1,  1],
        [ 1,  1,  1],
        [ 1,  1,  1]],

       ...,

       [[ 9,  5,  4],
        [ 9,  5,  4],
        [ 9,  5,  4],
        ...,
        [ 8,  4,  3],
        [ 7,  3,  2],
        [ 7,  3,  2]],

       [[ 8,  4,  3],
        [ 8,  4,  3],
        [ 8,  4,  3],
        ...,
        [ 7,  3,  2],
        [ 7,  3,  2],
        [ 7,  3,  2]],

       [[ 8,  4,  3],
        [ 8,  4,  3],
        [ 8,  4,  3],
        ...,
        [ 7,  3,  2],
        [ 7,  3,  2],
        [ 7,  3,  2]]], dtype=uint8)

In [759]:
result_a = cv2.imread('/Users/joheunkang/Desktop/comma/speed_challenge_2017/data/new_train_img/18061.jpg')
print(result_a)

[[[ 9  4  3]
  [ 9  4  3]
  [ 9  4  3]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 9  4  3]
  [ 9  4  3]
  [ 9  4  3]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[13  8  7]
  [13  8  7]
  [13  8  7]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 ...

 [[12  7  6]
  [12  7  6]
  [12  7  6]
  ...
  [ 9  5  4]
  [ 9  5  4]
  [ 9  5  4]]

 [[10  5  4]
  [10  5  4]
  [10  5  4]
  ...
  [ 9  5  4]
  [ 9  5  4]
  [ 9  5  4]]

 [[10  5  4]
  [10  5  4]
  [10  5  4]
  ...
  [ 9  5  4]
  [ 9  5  4]
  [ 9  5  4]]]


# Fitting Model (CNN)

In [760]:
TYPE_FLOW_PRECOMPUTED = 0
BATCH_SIZE = 128
EPOCH = 50

MODEL_NAME = 'CNNModel_flow'

download_path = '/Users/joheunkang/Desktop/comma/speed_challenge_2017/results'
weights_loc = os.path.join(download_path,'weights.h5')


In [761]:
model = CNNModel()

In [762]:
callbacks = [EarlyStopping(monitor='val_loss', patience=3),
             ModelCheckpoint(weights_loc , monitor='val_loss', save_best_only=True)]

In [763]:
history_object = model.fit_generator(training_generator,
                                     samples_per_epoch= len(train_samples)//BATCH_SIZE,
                                     validation_data=validation_generator,
                                     validation_steps=len(validation_samples)//BATCH_SIZE,
                                     callbacks=callbacks,
                                     epochs=EPOCH,
                                     verbose=1)


print('Training model complete...')

print(history_object.history.keys())
print('Loss')
print(history_object.history['loss'])
print('Validation Loss')
print(history_object.history['val_loss'])

/Users/joheunkang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., validation_steps=31, callbacks=[<keras.ca..., epochs=50, verbose=1, steps_per_epoch=127)`
  import sys


Epoch 1/50
127/127 [==============================] - 348s 3s/step - loss: 37.5062 - val_loss: 22.8797
Epoch 2/50
127/127 [==============================] - 363s 3s/step - loss: 16.3274 - val_loss: 14.4550
Epoch 3/50
127/127 [==============================] - 378s 3s/step - loss: 10.1265 - val_loss: 17.6550
Epoch 4/50
127/127 [==============================] - 384s 3s/step - loss: 5.3403 - val_loss: 14.4629
Epoch 5/50
127/127 [==============================] - 338s 3s/step - loss: 4.6402 - val_loss: 14.1090
Epoch 6/50
127/127 [==============================] - 354s 3s/step - loss: 5.1736 - val_loss: 15.6001
Epoch 7/50
127/127 [==============================] - 360s 3s/step - loss: 4.0950 - val_loss: 12.5910
Epoch 8/50
127/127 [==============================] - 384s 3s/step - loss: 3.3217 - val_loss: 10.5961
Epoch 9/50
127/127 [==============================] - 336s 3s/step - loss: 4.5883 - val_loss: 11.7786
Epoch 10/50
127/127 [==============================] - 326s 3s/step - loss: 2.9

__About 1.5 HOURS to train the model!!!__

# Training Result

loss = loss in traning data <br> 
val_loss== loss in validation data <br>

If there is a huge gap between loss and val_loss, we can think of it as __overfitting__.<br>. However, used earlystopping to avoid overfitting during the training. <br>

also, by adding <br> 

"save_best_only == True"

```python

ModelCheckpoint(weights_loc , monitor='val_loss', save_best_only=True)

```

we only saved weights that has smallest validation loss during our training.

which is <br> 

Validation_loss = __5.763716697692871__



# Prediction on test data 

We save the best weight as weight.h5 (lowest validation error).<br>

Here, we are using the weights to predict the test video.<br>


As we did for the traninig data, we first change the video to frames, and then to the optical flow frames.
Then, we take the the predicted speed from it. <br>

The result will be the __'test_labels.txt'__ <br>


In [766]:
MODEL_NAME = 'CNNModel_flow'
best_model_weights = os.path.join(download_path,'weights.h5')

In [772]:
def predict_from_video(video_input_path):
    predicted_labels = []

    video_reader = cv2.VideoCapture(video_input_path)

    num_frames = video_reader.get(cv2.CAP_PROP_FRAME_COUNT)
    frame_size = (int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fps = int(video_reader.get(cv2.CAP_PROP_FPS))

    t1 = time.time()
    ret, prev_frame = video_reader.read()
    hsv = np.zeros_like(prev_frame)

    predicted_labels.append(0.0)

    flow_image_bgr_prev1 =  np.zeros_like(prev_frame)
    flow_image_bgr_prev2 =  np.zeros_like(prev_frame)
    flow_image_bgr_prev3 =  np.zeros_like(prev_frame)
    flow_image_bgr_prev4 =  np.zeros_like(prev_frame)

    count =0
    while True:
        ret, next_frame = video_reader.read()
        if ret is False:
            break

        flow_image_bgr_next = convertToOptical(prev_frame, next_frame)
        flow_image_bgr = (flow_image_bgr_prev1 + flow_image_bgr_prev2 +flow_image_bgr_prev3 +flow_image_bgr_prev4 + flow_image_bgr_next)/4

        curr_image = cv2.cvtColor(next_frame, cv2.COLOR_BGR2RGB)

        combined_image = flow_image_bgr

        combined_image_test = cv2.normalize(combined_image, None, alpha=-1, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        combined_image_test = cv2.resize(combined_image_test, (0,0), fx=0.5, fy=0.5)

        combined_image_test = combined_image_test.reshape(1, combined_image_test.shape[0], combined_image_test.shape[1], combined_image_test.shape[2])

        prediction = model.predict(combined_image_test)

        predicted_labels.append(prediction[0][0])

        prev_frame = next_frame
        flow_image_bgr_prev4 = flow_image_bgr_prev3
        flow_image_bgr_prev3 = flow_image_bgr_prev2
        flow_image_bgr_prev2 = flow_image_bgr_prev1
        flow_image_bgr_prev1 = flow_image_bgr_next

        count +=1

    predicted_labels[0] = predicted_labels[1]
    return predicted_labels

# Get Predicted Labels

NOTE: it takes some times.

In [784]:
model = CNNModel()
model.load_weights(best_model_weights)
print('Your testing model is here!')
predicted_labels = predict_from_video(PATH_TEST_VIDEO)

Your testing model is here!


In [785]:
len(predicted_labels)

10798

we have correct number of predicted labels! 

# Result

In [786]:
with open(test_labels_result, mode="w") as outfile:
    for label in predicted_labels:
        outfile.write("%s\n" % np.str(label))

__Result of test_labels uploaded on GitHub__ <br>


## THANK YOU